In [1]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy --upgrade


  Using cached numpy-2.0.0-cp39-cp39-win_amd64.whl (16.5 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
tensorflow-intel 2.16.2 requires numpy<2.0.0,>=1.23.5; python_version <= "3.11", but you have numpy 2.0.0 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 2.0.0 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 2.0.0 which is incompatible.


In [6]:
!pip install tensorflow --upgrade


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.26.4 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.4 which is incompatible.



  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.0
    Uninstalling numpy-2.0.0:
      Successfully uninstalled numpy-2.0.0


In [3]:
!pip install tensorflow --upgrade


In [4]:
!pip uninstall keras
!pip install keras


^C


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import os

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# Load the CSV file
csv_file_path = 'path_to_your_csv_file.csv'
df = pd.read_csv(csv_file_path)

# Load the .npy files and add them to the dataframe
spectra = []
for filename in df['filename']:
    npy_file_path = os.path.join('np_data', filename + '.npy')
    spectrum = np.load(npy_file_path)
    spectra.append(spectrum)

# Add spectra to the dataframe
df['spectrum'] = spectra

# Extract features and target
features = df[['state', 'level', 'type', 'bullet votes']]
spectra = np.array(df['spectrum'].tolist())

# Normalize the spectra
scaler = MinMaxScaler()
spectra_normalized = scaler.fit_transform(spectra)

# Encode categorical variables
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(features[['state', 'level', 'type']]).toarray()

# Combine encoded categorical features with numerical features
bullet_votes = features[['bullet votes']].values
X = np.hstack((encoded_features, bullet_votes))

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, spectra_normalized, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='linear')
])

# Compile the model
model.compile(optimizer=Adam(), loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f'Model Loss: {loss}')

# Predict for a new input
new_input_features = pd.DataFrame({
    'state': ['new_state'],
    'level': ['new_level'],
    'type': ['new_type'],
    'bullet votes': [new_total_bullet_votes]
})

# Encode new input
new_encoded_features = encoder.transform(new_input_features[['state', 'level', 'type']]).toarray()
new_bullet_votes = new_input_features[['bullet votes']].values
new_input = np.hstack((new_encoded_features, new_bullet_votes))

# Predict the spectrum
predicted_spectrum = model.predict(new_input)
predicted_spectrum_denormalized = scaler.inverse_transform(predicted_spectrum)
print(predicted_spectrum_denormalized)
